In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
import matplotlib.pyplot as plt

In [12]:
def train_models(model_names, train_data_dir, test_data_dir, results_base_dir, models_base_dir, epochs=20, batch_size=32):
    # Load and prepare data
    print(f"Load training dataset from {train_data_dir}")
    train_data = tf.keras.utils.image_dataset_from_directory(
        train_data_dir,
        image_size=(224, 224)
    )
    
    print(f"Load testing dataset from {test_data_dir}")
    test_data = tf.keras.utils.image_dataset_from_directory(
        test_data_dir,
        image_size=(224, 224)
    )

    train_data_iterator = train_data.as_numpy_iterator()
    batch = train_data_iterator.next()

    # Split train/val
    train_size = int(len(train_data) * 0.8)
    val_size = int(len(train_data) * 0.2)
    train = train_data.take(train_size)
    val = train_data.skip(train_size).take(val_size)

    for model_name in model_names:
        print(f"Training {model_name}...")

        # Set up model-specific directories
        results_dir = os.path.join(results_base_dir, model_name)
        models_dir = os.path.join(models_base_dir, model_name)
        os.makedirs(results_dir, exist_ok=True)
        os.makedirs(models_dir, exist_ok=True)

        # Build model
        input_shape = (224, 224, 3)
        base_model = get_base_model(model_name, input_shape)
        model = build_model(base_model, input_shape)

        # Compile model
        model.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=keras.optimizers.RMSprop(learning_rate=1e-2),
            metrics=[keras.metrics.BinaryAccuracy()]
        )

        # Set up callbacks
        callbacks = get_callbacks(model_name, results_dir, models_dir)

        # Train model
        history = model.fit(
            train,
            validation_data=val,
            epochs=epochs,
            verbose=1,
            callbacks=callbacks
        )

        # Plot training history
        plot_training_history(history, model_name, results_dir)

        # Reduce learning rate and continue training
        model = load_model(os.path.join(models_dir, f"{model_name}_bo{epochs}_lr001.h5"))
        model.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
            metrics=[keras.metrics.BinaryAccuracy()]
        )

        callbacks = get_callbacks(model_name, results_dir, models_dir, lr_suffix="0001")
        history = model.fit(
            train,
            validation_data=val,
            epochs=epochs,
            verbose=1,
            callbacks=callbacks
        )

        plot_training_history(history, model_name, results_dir, lr_suffix="0001")

        # Test model
        test_model(model_name, test_data, models_dir)

In [3]:
def get_base_model(model_name, input_shape):
    model_dict = {
        'DenseNet121': tf.keras.applications.DenseNet121,
        'DenseNet169': tf.keras.applications.DenseNet169,
        'DenseNet201': tf.keras.applications.DenseNet201,
        'EfficientNetB0': tf.keras.applications.EfficientNetB0,
        'EfficientNetB1': tf.keras.applications.EfficientNetB1,
        'EfficientNetB2': tf.keras.applications.EfficientNetB2,
        'EfficientNetB3': tf.keras.applications.EfficientNetB3,
        'EfficientNetB4': tf.keras.applications.EfficientNetB4,
        'EfficientNetB5': tf.keras.applications.EfficientNetB5,
        'EfficientNetB6': tf.keras.applications.EfficientNetB6,
        'EfficientNetB7': tf.keras.applications.EfficientNetB7,
        'EfficientNetV2B0': tf.keras.applications.EfficientNetV2B0,
        'EfficientNetV2B1': tf.keras.applications.EfficientNetV2B1,
        'EfficientNetV2B2': tf.keras.applications.EfficientNetV2B2,
        'EfficientNetV2B3': tf.keras.applications.EfficientNetV2B3,
        'EfficientNetV2L': tf.keras.applications.EfficientNetV2L,
        'EfficientNetV2M': tf.keras.applications.EfficientNetV2M,
        'EfficientNetV2S': tf.keras.applications.EfficientNetV2S,
        'InceptionResNetV2': tf.keras.applications.InceptionResNetV2,
        'InceptionV3': tf.keras.applications.InceptionV3,
        'MobileNet': tf.keras.applications.MobileNet,
        'MobileNetV2': tf.keras.applications.MobileNetV2,
        'MobileNetV3Large': tf.keras.applications.MobileNetV3Large,
        'MobileNetV3Small': tf.keras.applications.MobileNetV3Small,
        'NASNetLarge': tf.keras.applications.NASNetLarge,
        'NASNetMobile': tf.keras.applications.NASNetMobile,
        'ResNet101': tf.keras.applications.ResNet101,
        'ResNet152': tf.keras.applications.ResNet152,
        'ResNet50': tf.keras.applications.ResNet50,
        'ResNet101V2': tf.keras.applications.ResNet101V2,
        'ResNet152V2': tf.keras.applications.ResNet152V2,
        'ResNet50V2': tf.keras.applications.ResNet50V2,
        'VGG16': tf.keras.applications.VGG16,
        'VGG19': tf.keras.applications.VGG19,
        'Xception': tf.keras.applications.Xception,
    }
    
    if model_name not in model_dict:
        raise ValueError(f"Unsupported model: {model_name}")
    
    return model_dict[model_name](input_shape=input_shape, include_top=False, weights='imagenet')

In [4]:
def build_model(base_model, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [5]:
def get_callbacks(model_name, results_dir, models_dir, lr_suffix="001"):
    log_file = os.path.join(results_dir, f"{model_name}_bo20_lr{lr_suffix}.csv")
    return [
        CSVLogger(log_file),
        ModelCheckpoint(
            filepath=os.path.join(models_dir, f"{model_name}_bo20_lr{lr_suffix}.h5"),
            save_weights_only=False,
            save_best_only=True,
            save_freq='epoch',
            verbose=1
        )
    ]

In [6]:
def plot_training_history(history, model_name, results_dir, lr_suffix="001"):
    plt.figure()
    plt.plot(history.history['loss'], color='teal', label='loss')
    plt.plot(history.history['val_loss'], color='orange', label='val_loss')
    plt.title(f'{model_name} Loss (LR: 0.{lr_suffix})')
    plt.legend(loc="upper left")
    plt.savefig(os.path.join(results_dir, f"{model_name}_loss_lr{lr_suffix}.png"))
    plt.close()

In [7]:
def test_model(model_name, test_data, models_dir):
    model = tf.keras.models.load_model(os.path.join(models_dir, f"{model_name}_bo20_lr0001.h5"))
    pre = Precision()
    re = Recall()
    acc = BinaryAccuracy()

    for batch in test_data.as_numpy_iterator():
        X, y = batch
        yhat = model.predict(X)
        pre.update_state(y, yhat)
        re.update_state(y, yhat)
        acc.update_state(y, yhat)

    f1_score = 2 * (pre.result() * re.result()) / (pre.result() + re.result())
    print(f"{model_name} Test Results:")
    print(f"Precision: {pre.result():.4f}")
    print(f"Recall: {re.result():.4f}")
    print(f"Accuracy: {acc.result():.4f}")
    print(f"F1 Score: {f1_score:.4f}")


In [13]:
# Usage example:
model_to_train = ['DenseNet201', 'EfficientNetB7', 'EfficientNetV2L']
train_data_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px"
test_data_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px_test"
results_base_dir = r"D:\Kananat\_result\log"
models_base_dir = r"D:\Kananat\_result\model"
train_models(model_to_train, train_data_dir, test_data_dir, results_base_dir, models_base_dir,epochs=20, batch_size=1)

Load training dataset from D:\Kananat\TF_TMJOA_jpg_x_5px
Found 34203 files belonging to 2 classes.
Load testing dataset from D:\Kananat\TF_TMJOA_jpg_x_5px_test
Found 7205 files belonging to 2 classes.
Training DenseNet201...
Epoch 1/20


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[32,14,14,1632] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_3/densenet201/conv4_block43_concat/concat-0-0-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_243593]